In [1]:
# Import standard libraries
import os
from importlib import reload
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import pandas as pd
import pickle
import itertools
from itertools import groupby
import os.path
import math
import pybedtools
import time
from tqdm import tqdm
import random
import MOODS.parsers
import MOODS.tools
import MOODS.scan
import subprocess
# Custom libraries
import utils as lu
import process_jaspar_with_cage as pj
# Reload modules in case of modifications
reload(lu)
reload(pj)

/home/louiscam/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


<module 'process_jaspar_with_cage' from '/home/louiscam/projects/gpcr/code/JASPAR_processing/process_jaspar_with_cage.py'>

# Directories

In [2]:
# Directory of adhesome data
dir_adhesome = '/home/louiscam/projects/gpcr/data/adhesome_data/'
# Directory of genome data
dir_genome = '/home/louiscam/projects/gpcr/data/genome_data/'
prom_hg19_seq_dir = dir_genome+'prom_hg19_seq_dir/'
# Directory of processed HiC
dir_processed_hic = '/home/louiscam/projects/gpcr/save/processed_hic_data_dir/'
# Directory for storing preliminary results
prelim_results_dir = '/home/louiscam/projects/gpcr/save/prelim_results_dir/'
# Directory of epigenomic data
epigenome_dir = '/home/louiscam/projects/gpcr/data/epigenome_data/'
processed_epigenome_data_dir = '/home/louiscam/projects/gpcr/save/processed_epigenome_data_dir/'
# Saving directory
saving_dir = '/home/louiscam/projects/gpcr/save/figures/'
# Directory of tf data
tf_dir = '/home/louiscam/projects/gpcr/data/tf_data/'
jaspar_dir = tf_dir+'jaspar_data/'
pfm_dir = tf_dir+'pfm_data/'
moods_out_dir = tf_dir+'moods_cage_outdir/'
fantom5_dir = '/home/louiscam/projects/gpcr/data/fantom5/'
cage_fasta_dir = dir_genome+'cage_hg19_seq_dir/'

# 1. Select genes of interest

### Load genes of interest

In [3]:
# Genes of interest and location
adh_tf_genes = pickle.load(open(saving_dir+'adh_tf_genes.pkl', 'rb'))
print('Number of genes of interest = '+str(len(adh_tf_genes)))

Number of genes of interest = 339


### Gene location

In [4]:
# Load correspondance between UCSC gene name and HGNC gene name
gene_id_filename = dir_genome+'chrom_hg19.name'
df_name0 = pd.read_csv(gene_id_filename, sep = '\t', header = 0,dtype={"rfamAcc": str, "tRnaName": str})
# Only keep UCSC name and geneSymbol
df_name0 = df_name0[['#kgID','geneSymbol']]
df_name0.columns = ['#name','geneSymbol']
df_name0['geneSymbol'] = df_name0['geneSymbol'].str.upper()
# Mapping UCSC to HGNC
ucsc_to_hgnc = pj.create_ucsc_hgnc_dict(dir_genome+'chrom_hg19.name')

In [6]:
# Load gene locations
gene_locations_filename = dir_genome+'chrom_hg19.loc'
df_loc0 = pd.read_csv(gene_locations_filename, sep = '\t', header = 0)
# Merge df_loc0 and df_name0
df_loc1 = pd.merge(df_name0, df_loc0, on=['#name'])
# Filter out irregular chromosomes
keep_chrom = ['chr'+str(i) for i in np.arange(1,23)]
df_loc2 = df_loc1[df_loc1['chrom'].isin(keep_chrom)]
df_loc2 = df_loc2.sort_values(by=['geneSymbol'])
# Remove ribosomal RNA (may be able to remove more irrelevant things, how?)
df_loc3 = df_loc2[~df_loc2['geneSymbol'].str.contains('RRNA')]
# Restrict to genes of interest
df_loc4 = df_loc3[df_loc3['geneSymbol'].isin(adh_tf_genes)]
df_loc4.columns = ['ucsc_gene','hgnc_gene','chrom','strand','txStart','txEnd','cdsStart','cdsEnd','exonCount',
                  'exonStarts','exonEnds','proteinID','alignID']

In [10]:
# Define region 1kb upstream of transcripts
df_loc4['upstream_start'] = [df_loc4['txStart'].iloc[i]-1000 if (df_loc4['strand'].iloc[i]=='+') 
                             else df_loc4['txEnd'].iloc[i] for i in range(df_loc4.shape[0])]
df_loc4['upstream_end'] = [df_loc4['txStart'].iloc[i] if (df_loc4['strand'].iloc[i]=='+') 
                             else df_loc4['txEnd'].iloc[i]+1000 for i in range(df_loc4.shape[0])]

In [15]:
# Visualize location dataframe
df_loc = df_loc4.copy()
print('Number of unique UCSC transscripts = '+str(len(np.unique(df_loc['ucsc_gene']))))
df_loc.head()

Number of unique UCSC transscripts = 1722


,ucsc_gene,hgnc_gene,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,proteinID,alignID,upstream_start,upstream_end
9625,uc010qdg.2,ABI1,chr10,-,27035524,27066170,27037498,27066068,7,"27035524,27040526,27047990,27054146,27057780,2...","27037674,27040712,27048167,27054247,27057921,2...",B4DKX2,uc010qdg.2,27066170,27067170
9620,uc010qdk.2,ABI1,chr10,-,27035524,27150016,27037498,27149792,9,"27035524,27040526,27054146,27057780,27059173,2...","27037674,27040712,27054247,27057921,27059274,2...",B6VEX4,uc010qdk.2,27150016,27151016
9623,uc010qdh.2,ABI1,chr10,-,27035524,27150016,27037498,27149792,8,"27035524,27040526,27054146,27057780,27059173,2...","27037674,27040712,27054247,27057921,27059274,2...",Q8IZP0-10,uc010qdh.2,27150016,27151016
9613,uc001itb.3,ABI1,chr10,-,27035524,27150016,27037498,27149792,12,"27035524,27040526,27047990,27052808,27054146,2...","27037674,27040712,27048167,27052889,27054247,2...",H7BXI6,uc001itb.3,27150016,27151016
9616,uc001isy.3,ABI1,chr10,-,27035524,27150016,27037498,27149792,11,"27035524,27040526,27044583,27047990,27054146,2...","27037674,27040712,27044670,27048167,27054247,2...",Q8IZP0-9,uc001isy.3,27150016,27151016


# 2. Determine CAGE peaks within 1kb upstream of genes of interest

### Process robust CAGE peaks

In [16]:
# Load annotated FANTOM5 CAGE peaks
cage_df = pd.read_csv(fantom5_dir+'hg19.cage_peak_phase1and2combined_ann.txt', sep='\t', skiprows=7, header=0)
# Define new columns
cage_df['chrom'] = cage_df['00Annotation'].str.split(':', expand=True)[0]
cage_df['cage_start'] = cage_df['00Annotation'].str.split(':', expand=True)[1].str.split(",", expand=True)[0].str.split(".", expand=True)[0].astype(int)
cage_df['cage_end'] = cage_df['00Annotation'].str.split(':', expand=True)[1].str.split(",", expand=True)[0].str.split(".", expand=True)[2].astype(int)
cage_df['strand'] = cage_df['00Annotation'].str.split(':', expand=True)[1].str.split(",", expand=True)[1]
cage_df['peak_id'] = cage_df['short_description'].str.split('@', expand=True)[0].str.strip('p')
# Reformat dataframe
cage_df = cage_df[['short_description','association_with_transcript','chrom','cage_start','cage_end',
                   'strand','peak_id']]
cage_df = cage_df.sort_values(by=['chrom','cage_start'])
print('Number of unique CAGE peaks = '+str(len(np.unique(cage_df['short_description'].values))))
cage_df.head()

Number of unique CAGE peaks = 201802


,short_description,association_with_transcript,chrom,cage_start,cage_end,strand,peak_id
95563,p1@MTND1P23,129bp_to_ENST00000416931_5end,chr1,564571,564600,+,1
95564,p3@MTND1P23,197bp_to_ENST00000416931_5end,chr1,564639,564649,+,3
95565,p3@MTND2P28,246bp_to_ENST00000457540_5end,chr1,565266,565278,+,3
95566,p4@MTND2P28,458bp_to_ENST00000457540_5end,chr1,565478,565483,+,4
95567,p1@MTND2P28,489bp_to_ENST00000457540_5end,chr1,565509,565541,+,1


### CAGE peaks within 1kb of genes of interest

In [17]:
# Create bed file for regions upstream of genes of interest
upstream_gene_df = df_loc[['chrom','upstream_start','upstream_end','ucsc_gene']]
upstream_gene_df.columns = ['chrom','start','stop','name']
upstream_gene_bed = pybedtools.BedTool.from_dataframe(upstream_gene_df).sort()

In [18]:
# Create a bed file for robust CAGE peaks
cage_peak_df = cage_df[['chrom','cage_start','cage_end','short_description']]
cage_peak_df.columns = ['chrom','start','end','name']
cage_bed = pybedtools.BedTool.from_dataframe(cage_peak_df).sort()

In [19]:
# Get CAGE peaks for each gene of interest
out = pybedtools.bedtool.BedTool.map(upstream_gene_bed, cage_bed, c = 4, o = 'distinct')
out_df = out.to_dataframe()
out_df.columns=['chrom','upstream_start','upstream_end','ucsc_gene','peaks']

In [20]:
# Reformat matching results
peaks_per_gene_df = pd.merge(df_loc, out_df, on=['chrom','upstream_start','upstream_end','ucsc_gene'])
peaks_per_gene_df = peaks_per_gene_df[peaks_per_gene_df['peaks'] != '.']
peaks_per_gene_df['peaks'] = peaks_per_gene_df['peaks'].str.split(',')

In [23]:
# Visualize results
peaks_per_gene_df[['ucsc_gene', 'hgnc_gene', 'chrom', 'strand', 'txStart', 'txEnd',
        'cdsStart', 'cdsEnd', 'exonCount', 'upstream_start', 'upstream_end','peaks']].head()

,ucsc_gene,hgnc_gene,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,upstream_start,upstream_end,peaks
14,uc010zij.2,ABI2,chr2,+,204193002,204296892,204240738,204292075,10,204192002,204193002,[p1@ABI2]
15,uc010zii.2,ABI2,chr2,+,204193002,204296892,204193237,204292075,10,204192002,204193002,[p1@ABI2]
16,uc002uzz.3,ABI2,chr2,+,204193002,204296892,204193237,204292075,10,204192002,204193002,[p1@ABI2]
18,uc010zin.2,ABI2,chr2,+,204259422,204296892,204267321,204292075,5,204258422,204259422,[p7@ABI2]
19,uc010zih.2,ABI2,chr2,+,204193002,204296892,204267321,204292075,5,204192002,204193002,[p1@ABI2]


# 3. Associate CAGE peaks with genes

In [25]:
# Dataframe of genes for each peak
genes_per_peak = peaks_per_gene_df[['ucsc_gene','peaks']]
genes_per_peak = lu.unnesting(genes_per_peak,['peaks'])
genes_per_peak = genes_per_peak[genes_per_peak['peaks'].str.contains('p')]
genes_per_peak = genes_per_peak.groupby('peaks')['ucsc_gene'].apply(list).reset_index(name='ucsc_genes')
genes_per_peak.columns = ['short_description','ucsc_genes']

In [26]:
# Add peak location
genes_per_peak = pd.merge(genes_per_peak, cage_df, on=['short_description'])
genes_per_peak = genes_per_peak[['short_description','chrom','cage_start','cage_end','strand','ucsc_genes']]
genes_per_peak = genes_per_peak.sort_values(by=['chrom'])

In [27]:
# Consider a window around each peak
genes_per_peak['window_start'] = [genes_per_peak['cage_start'].iloc[i]-400 if (genes_per_peak['strand'].iloc[i]=='+') 
                                  else genes_per_peak['cage_start'].iloc[i]-50 for i in range(genes_per_peak.shape[0])]
genes_per_peak['window_end'] = [genes_per_peak['cage_end'].iloc[i]+50 if (genes_per_peak['strand'].iloc[i]=='+') 
                                  else genes_per_peak['cage_end'].iloc[i]+400 for i in range(genes_per_peak.shape[0])]

In [28]:
# Visualize
genes_per_peak.head()

,short_description,chrom,cage_start,cage_end,strand,ucsc_genes,window_start,window_end
672,p2@SLC9A1,chr1,27481950,27482009,-,"[uc001bnm.4, uc010ofk.3, uc001bnn.3]",27481900,27482409
698,p2@USF1,chr1,161015752,161015778,-,"[uc001fxj.4, uc001fxi.4]",161015702,161016178
798,p3@HDAC1,chr1,32757519,32757558,+,"[uc001bvb.1, uc010ohe.1, uc010ohd.1, uc010ohf.1]",32757119,32757608
138,p15@HDGF,chr1,156722252,156722268,-,"[uc001fpz.4, uc010phr.2, uc009wse.3, uc001fpy.4]",156722202,156722668
1111,p5@SHC1,chr1,154943232,154943245,-,"[uc001ffw.3, uc001ffv.3]",154943182,154943645


# 4. Create sequence files for all CAGE windows

# 5. Search for TFBS in all CAGE windows
See process_jaspar_with_cage.py

# 6. Find TFs binding each CAGE window

### Useful data

In [29]:
# Load JASPAR TFBS metadata
jaspar_df = pd.read_csv(tf_dir+'JASPAR-HomoSapiens.csv', 
                        header=0, usecols=['ID','Name','Species','Class','Family'])
# Mapping JASPAR ID to HGNC
jasparid_to_hgnc = {jaspar_df.iloc[i,0]:jaspar_df.iloc[i,1] for i in range(jaspar_df.shape[0])}

In [31]:
# Visualize data
print('Number of unique TFs = '+str(len(np.unique(jaspar_df['Name']))))
print('Number of unique TF motifs = '+str(len(np.unique(jaspar_df['ID']))))
jaspar_df.head()

Number of unique TFs = 639
Number of unique TF motifs = 810


,ID,Name,Species,Class,Family
0,MA0002.1,RUNX1,Homo sapiens,Runt domain factors,Runt-related factors
1,MA0003.1,TFAP2A,Homo sapiens,Basic helix-span-helix factors (bHSH),AP-2
2,MA0003.2,TFAP2A,Homo sapiens,Basic helix-span-helix factors (bHSH),AP-2
3,MA0003.3,TFAP2A,Homo sapiens,Basic helix-span-helix factors (bHSH),AP-2
4,MA0003.4,TFAP2A,Homo sapiens,Basic helix-span-helix factors (bHSH),AP-2


### Load MOODS results

In [37]:
# Create dictionary mapping CAGE peaks to the corresponding TFs
tfs_associated_to_cage_peaks = {}

In [39]:
peak_moods_file = os.listdir(moods_out_dir)[0]
# load results dataframe for that gene
cage_peak = peak_moods_file.strip('.txt')
res_df = pd.read_csv(moods_out_dir+peak_moods_file, sep=',', header=None, 
                     usecols=[0,1,2,3,4,5], 
                     names=['peak_id','tfbs_file','motif_loc','strand','score','motif'])
# Add columns corresponding to the target gene
res_df['cage_peak'] = cage_peak
res_df['chrom'] = res_df['peak_id'].str.split(':', expand=True)[0]
location_cols = res_df['peak_id'].str.split(':', expand=True)[1]
res_df[['start','stop']] = location_cols.str.split('-', expand=True)
# Add columns corresponding to the TFBS
res_df['jaspar_tfbs'] = res_df['tfbs_file'].str.strip('.pfm')
res_df['TF'] = [jasparid_to_hgnc[jaspar_id] for jaspar_id in res_df['jaspar_tfbs']]
# Select relevant columns
res_df = res_df[['cage_peak', 'chrom','start','stop','motif_loc',
                 'jaspar_tfbs','TF','score','strand','motif']]
# Store list of TFs associated with the current CAGE peak in dict
tfs_associated_to_cage_peaks[cage_peak] = res_df['TF'].values

# 7. Find TFs targeting genes of interest

In [40]:
# Construct dataframe with gene-tf pairs
tf_and_genes_df = genes_per_peak.copy()
tf_and_genes_df['tfs'] = [tfs_associated_to_cage_peaks[cage_peak] if cage_peak in tfs_associated_to_cage_peaks.keys()
                          else ['ToDo']
                          for cage_peak in tf_and_genes_df['short_description'].values]
tf_and_genes_df = lu.unnesting(tf_and_genes_df,['ucsc_genes'])
tf_and_genes_df = lu.unnesting(tf_and_genes_df,['tfs'])
tf_and_genes_df['hgnc_gene'] = [ucsc_to_hgnc[ucsc_id] for ucsc_id in tf_and_genes_df['ucsc_genes']]
tf_and_genes_df = tf_and_genes_df[['ucsc_genes','hgnc_gene','tfs',
                                   'short_description','chrom','cage_start','cage_end','strand']]

/home/louiscam/projects/gpcr/code/JASPAR_processing/utils.py:37: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  return df1.join(df.drop(explode, 1), how='left')


In [41]:
# Visualize results
tf_and_genes_df.sort_values('short_description').head()

,ucsc_genes,hgnc_gene,tfs,short_description,chrom,cage_start,cage_end,strand
0,uc002lga.3,TCF4,ToDo,p101@TCF4,chr18,53303180,53303190,-
0,uc021ukp.1,TCF4,ToDo,p101@TCF4,chr18,53303180,53303190,-
0,uc002lga.3,TCF4,ToDo,p101@TCF4,chr18,53303180,53303190,-
0,uc021ukp.1,TCF4,ToDo,p101@TCF4,chr18,53303180,53303190,-
1,uc011avz.1,ANKRD28,ToDo,p10@ANKRD28,chr3,15839693,15839707,-
